In [3]:
import pandas as pd
import json
import csv
import sys
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
import string
import numpy as np
from nltk.corpus import stopwords
import string
import emoji
import regex
import string
from datetime import timedelta
import matplotlib.dates as mdates
import seaborn as sn
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from unidecode import unidecode as unidec

In [4]:
dir_="C:\\Users\\Alex\\Personal-Code\\ACE_592_Dating_Sim\\"

In [21]:
dfA = pd.read_csv(dir_ + "Raw_Data\\Amazon_reviews.csv")

In [22]:
dfA.head(10)

,score,review_text
0,5.0,I initially had trouble deciding between the p...
1,5.0,Allow me to preface this with a little history...
2,4.0,I am enjoying it so far. Great for reading. Ha...
3,5.0,I bought one of the first Paperwhites and have...
4,5.0,I have to say upfront - I don't like coroporat...
5,NaN,"My previous kindle was a DX, this is my second..."
6,NaN,Allow me to preface this with a little history...
7,NaN,Just got mine right now. Looks the same as the...
8,NaN,I initially had trouble deciding between the p...
9,NaN,I am enjoying it so far. Great for reading. Ha...


In [23]:
dfA.shape

(4037, 2)

In [28]:
#cleans and crops the data. Prepares text for sentiment analysis
def clean(df1):
    #throws the copy error warning because it annoys me
    pd.options.mode.chained_assignment = None  # default='warn'

    #culls any review that does not have a score or a review text and stores the new df as df
    df2 = df1.loc[df1['score'] != 0]
    df3 = df2.dropna(subset = ['score']).reset_index()
    df = df3.dropna(subset = ['review_text']).reset_index()
    
    #CLEANING THE TEXT (scrub scrub)
    #converts text to string for all review comments 
    df['full_text'] = df['review_text'].astype(str)
    
    #converts all text to lowercase: 
    df['clean_text'] =  df['full_text'].str.lower().str.replace("\n","")
    
    #removes punctuation
    string.punctuation
    # This maketrans function can be used to remove a list of characters from a string.
    remv_punc = str.maketrans('','',string.punctuation + '“' +"‘"+'”')
    df['clean_text'] = df['clean_text'].str.translate(remv_punc)
    
    #on the off chance that any of these come up in the reviews
    #starts by removing the https, #, and @ AND THEIR FOLLOWING TEXT from our full text tweets 
    #Regex functions to find each one
    find_links = re.compile(r'http\S+')
    find_users = re.compile(r'@\w+')
    find_hashtags = re.compile(r'#\w+')

    #"sub" them for an empty string
    df['clean_text'] = [find_links.sub("",x) for x in df['clean_text']]
    df['clean_text']=  [find_users.sub("",x) for x in df['clean_text']]
    df['clean_text']= [find_hashtags.sub("",x) for x in df['clean_text']]
    
    
    return df

In [29]:
df_A = clean(dfA)
df_A.shape
df_A.head(5);

(3617, 6)

In [32]:
#verification
import nltk
nltk.download('stopwords');
stopwords.words('english');
from nltk.corpus import stopwords;
stopwords.words('english');
print (stopwords.words('english'));

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
def get_sent(df):
    sid = SentimentIntensityAnalyzer()

    #takes the clean text and encodes it in utf-8, otherwise Vader will be most unpleased
    clean = df['clean_text']
    clean = str(clean).encode('utf-8')

    #calculates the sentiment of each review and stores the results in the 
    df1 = df.copy()
    df1['comp_sent'] = df['clean_text'].apply(lambda clean:sid.polarity_scores(str(clean))['compound'])
    df1['neg_sent'] = df['clean_text'].apply(lambda clean:sid.polarity_scores(str(clean))['neg'])
    df1['pos_sent'] = df['clean_text'].apply(lambda clean:sid.polarity_scores(str(clean))['pos'])
    df1['neu_sent'] = df['clean_text'].apply(lambda clean:sid.polarity_scores(str(clean))['neu'])
    
    
    return df1

In [34]:
#computes the sentiment scores for the amazon reviews and stores the scores in new columns
df_A = get_sent(df_A)
df_A.head(5)

,level_0,index,score,review_text,full_text,clean_text,comp_sent,neg_sent,pos_sent,neu_sent
0,0,0,5.0,I initially had trouble deciding between the p...,I initially had trouble deciding between the p...,i initially had trouble deciding between the p...,0.9799,0.038,0.168,0.794
1,1,1,5.0,Allow me to preface this with a little history...,Allow me to preface this with a little history...,allow me to preface this with a little history...,0.9868,0.041,0.146,0.814
2,2,2,4.0,I am enjoying it so far. Great for reading. Ha...,I am enjoying it so far. Great for reading. Ha...,i am enjoying it so far great for reading had ...,0.4364,0.181,0.223,0.596
3,3,3,5.0,I bought one of the first Paperwhites and have...,I bought one of the first Paperwhites and have...,i bought one of the first paperwhites and have...,0.9734,0.030,0.104,0.866
4,4,4,5.0,I have to say upfront - I don't like coroporat...,I have to say upfront - I don't like coroporat...,i have to say upfront i dont like coroporate ...,0.9922,0.095,0.195,0.710


In [41]:
def get_wc(df1):
    #throws the copy error warning because it annoys me
    pd.options.mode.chained_assignment = None  # default='warn'
    
    #drops the columns without text. We won't be using them for the RF since they can't return sentiment scores
    #df1 = df.dropna(subset = ['clean_text']).reset_index()
    #adds a WC column to the dataframe    
    df1['word_count'] = np.zeros(len(df1))
    
    
    #Crappy for loop to do this but it's whatever because we have no shortage of time
    #runs through the clean text in each row and counts the num of characters. Stores that value in a new df 
    #which will be merged with the existing df
    for i in range(len(df1)):
        wc = len(df1['clean_text'][i])
        df1['word_count'][i] = wc
    
    df2 = df1[['full_text','clean_text', 'score', 'comp_sent', 'neg_sent', 'pos_sent', 'neu_sent', 'word_count']]
    
    return df2

In [43]:
df_A_wc = get_wc(df_A)
df_A_wc.head(5)

,full_text,clean_text,score,comp_sent,neg_sent,pos_sent,neu_sent,word_count
0,I initially had trouble deciding between the p...,i initially had trouble deciding between the p...,5.0,0.9799,0.038,0.168,0.794,1030.0
1,Allow me to preface this with a little history...,allow me to preface this with a little history...,5.0,0.9868,0.041,0.146,0.814,1382.0
2,I am enjoying it so far. Great for reading. Ha...,i am enjoying it so far great for reading had ...,4.0,0.4364,0.181,0.223,0.596,176.0
3,I bought one of the first Paperwhites and have...,i bought one of the first paperwhites and have...,5.0,0.9734,0.030,0.104,0.866,1633.0
4,I have to say upfront - I don't like coroporat...,i have to say upfront i dont like coroporate ...,5.0,0.9922,0.095,0.195,0.710,1854.0


In [44]:
#saves the new CSV files as clean data
df_A_wc.to_csv(dir_ + "Wrangled_Data\\sent_Amazon.csv")